In [ ]:
import pandas as pd
file = "data/4import_Database_Austria_March2018.xlsx"
df = pd.read_excel(file, "RESEARCH")[2:]

In [ ]:
df.fillna(value=False, inplace=True)

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    if row['*SiteID']:
        temp_research, _ = ResearchEvent.objects.get_or_create(
            identifier=row['*Research Activity ID']
        )
        temp_research.name = row['*Research Activity ID']
        temp_research.start_date=row['Start Date']
        temp_research.end_date=row['End Date']
        if row['Comment']:
            temp_research.comment=row['Comment']
        
        if row['Responsible Researcher']:
            for x in row['Responsible Researcher'].split(','):
                if x:
                    temp_pers, _ = Person.objects.get_or_create(
                        written_name=x
                    )
                    temp_research.responsible_researcher.add(temp_pers)
        if row['Responsible Institution']:
            for x in row['Responsible Institution'].split(','):
                if x:
                    temp_inst, _ = Institution.objects.get_or_create(
                        written_name=x
                    )
                    temp_research.responsible_institution.add(temp_inst)
        if row['Public'] == "yes":
            temp_research.public = True
        temp_research.save()
        if row['Research Method']:
            temp_method,_ = SkosConcept.objects.get_or_create(
                pref_label=row['Research Method']
            )
            temp_research.research_method.add(temp_method)
        
        # hook to site
        temp_site, _ = Site.objects.get_or_create(
            identifier=row['*SiteID']
        )
        temp_site.information_source.add(temp_research)
        temp_site.save()
        temp_research.save()